Data Wrangle OpenStreetMaps Data
=================

Step One - Finish Lession 6
-------------------------

Done.

Step Two - Review the Rubric and Sample Project
--------------------------------------------

Done.

Step Three - Choose Your Map Area
-------------------------------

We use [Map Zen](https://mapzen.com/data/metro-extracts) to download the preselected metro area of San Francisco, California. After uncompressing, the dataset size is 648.9MB.

Step Four - Process your Dataset
------------------------------

### 4.1 Audit the dataset

#### Sanity check on element types

As a sanity check, we go through the dataset to see list all element types, which should contain the following:
  * `osm`: top-level root node
  * `node`, `way` and `relation`: instances of data primitives
  * `tag`: a general purpose node for key/value pair.
  * `nd`: used inside `way`s to reference a `node` element.
  * `member`: used inside a `relation`.

In [1]:
import xml.etree.cElementTree as ET

elem_types = set()
for event, elem in ET.iterparse("data/san-francisco_california.osm"):
    if elem.tag not in elem_types:
        elem_types.add(elem.tag)

print elem_types

set(['node', 'nd', 'bounds', 'member', 'tag', 'relation', 'way', 'osm'])


The only "unexpected" element type is `bounds`, which occurs only ones in the dataset to indicate the bounding box of this dataset.

### 4.2 clean the dataset

### 4.3 Convert from XML to JSON

### 4.4 Import into a MangoDB database and run queries

Step Five -Document your Work
---------------------------